In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import math
import numpy as np
import gc

## CNN

In [2]:
seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)
torch.backends.cudnn.deterministic = True

In [3]:
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

class CustomDataset(Dataset):
    def __init__(self, data, labels, device):
        self.data = torch.Tensor(data).to(device)
        self.labels = torch.Tensor(labels).to(device)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = {'data': self.data[idx], 'label': self.labels[idx]}
        return sample

# load data 
eeg_data = np.load('scaled_eeg_data.npy')
eeg_labels = np.load('scaled_eeg_labels.npy')

# Perform train-test split
test_size = 0.1  # adjust the test_size as needed
random_state = 42  # set random_state for reproducibility
train_data, test_data, train_labels, test_labels = train_test_split(eeg_data, eeg_labels, test_size=test_size, random_state=random_state)

del eeg_data, eeg_labels
gc.collect()

22

In [4]:
NUM_EPOCHS = 1000
BATCH_SIZE = 256
LR = 1e-3
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f'use device: {DEVICE}')

train_dataset = CustomDataset(train_data, train_labels, DEVICE)
test_dataset = CustomDataset(test_data, test_labels, DEVICE)

# Create DataLoader instances for training and testing
shuffle_train = True  # Set to True if you want to shuffle the training data
shuffle_test = False  # No need to shuffle the test data
num_workers = 2  # Number of parallel processes to load data

train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=shuffle_train, num_workers=num_workers)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=shuffle_test, num_workers=num_workers)

use device: cuda:0


In [5]:
class TimeSeriesCNN(nn.Module):
    def __init__(self, num_channels, sequence_length, num_classes):
        super(TimeSeriesCNN, self).__init__()
        
        # Define convolutional layers
        self.conv1 = nn.Conv1d(in_channels=num_channels, out_channels=4, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv1d(in_channels=4, out_channels=4, kernel_size=3, stride=1, padding=1)
        # self.conv3 = nn.Conv1d(in_channels=4, out_channels=4, kernel_size=3, stride=1, padding=1)
        
        # Pooling layer
        self.pool = nn.MaxPool1d(kernel_size=2, stride=2)
        
        # Calculate the input size for the fully connected layer
        fc_input_size = int(4  * (sequence_length / 4))  # Assuming two pooling layers with kernel size 2
        
        # Fully connected layers
        self.fc1 = nn.Linear(fc_input_size, 16)
        self.fc2 = nn.Linear(16, num_classes)

    def forward(self, x):
        # Input shape: (batch_size, num_channels, sequence_length)
        
        # Convolutional layers with ReLU activation and pooling
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        # x = F.relu(self.conv3(x))
        # x = self.pool(x)
        
        # Reshape for fully connected layers
        x = x.view(x.size(0), -1)
        
        # Fully connected layers with ReLU activation
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        
        return x

# Example usage:
# Assume you have 3 channels of time series data, each with a sequence length of 100, and 5 output classes
num_channels = 20
sequence_length = 1000
num_classes = 6

model = TimeSeriesCNN(num_channels, sequence_length, num_classes).to(DEVICE)
print(model)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LR)
criterion = criterion.to(DEVICE)


TimeSeriesCNN(
  (conv1): Conv1d(20, 4, kernel_size=(3,), stride=(1,), padding=(1,))
  (conv2): Conv1d(4, 4, kernel_size=(3,), stride=(1,), padding=(1,))
  (pool): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=1000, out_features=16, bias=True)
  (fc2): Linear(in_features=16, out_features=6, bias=True)
)


C:\Users\marton\AppData\Roaming\Python\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
#get the number of model parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 16,414 trainable parameters


In [7]:
train_data.shape

(95739, 1000, 20)

In [8]:
# train cnn model

seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)
torch.backends.cudnn.deterministic = True

loss_list = []
best_loss = np.inf



for epoch in range(NUM_EPOCHS):
    model.train()
    print(f'Epoch:{epoch + 1}:')
    # Wrap train_dataloader with tqdm for a progress bar
    for i, batch in enumerate(train_dataloader):
        x, y = batch['data'], batch['label']
        x = x.transpose(-1, -2)
        
        optimizer.zero_grad()

        y_pred = model(x)

        loss = criterion(y_pred, y)

        loss.backward()
        optimizer.step()

        loss_list.append(loss.detach().item())
        # if len(loss_list) > 100:
        #     print(f'Loss: {np.mean(loss_list[-100:])}')

        del loss, y_pred, x, y

    # After each epoch, perform validation and print accuracy
    # if ((epoch + 1) % 2) == 0:
    train_loss_list = []
    test_loss_list = []
    
    with torch.no_grad():
        model.eval()
        for i, batch in enumerate(train_dataloader):
            x, y = batch['data'], batch['label']
            x = x.transpose(-1, -2)

            y_pred = model(x.to(DEVICE))
            loss = criterion(y_pred, y.to(DEVICE))
            train_loss_list.append(loss.detach().item())

            del loss, y_pred, x, y
            gc.collect()

        for i, batch in enumerate(test_dataloader):
            x, y = batch['data'], batch['label']
            x = x.transpose(-1, -2)

            y_pred = model(x.to(DEVICE))
            loss = criterion(y_pred, y.to(DEVICE))
            test_loss_list.append(loss.detach().item())

            del loss, y_pred, x, y
            gc.collect()
        
        train_loss = np.mean(train_loss_list)
        test_loss = np.mean(test_loss_list)
        print(f'Train Loss: {train_loss}')
        print(f'Test Loss: {test_loss}')

    if test_loss < best_loss:
        best_loss = test_loss
        torch.save(model, f'saved_models/BEST_ep{epoch}.pkl')

Epoch:1:


RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
